# Prometheus Relation Model

`sbt publishM2`

Repeat if the project has been updated, and also do:

`rm -rf ~/.m2/repository/sonymobile`

`rm -rf metastore_db`

(*Important*: `kernel -> restart all` otherwise the previously used version of the promethues-relation-model lib be used again! Very confusing!) <- not sure if this is correct

`prometheus-relation-model` requires [docforia](https://github.com/marcusklang/docforia), which must be git-cloned and `mvn install`:ed first (not yet on Maven repository unfortunately).

In [2]:
%AddDeps sonymobile prometheus-relation-model_2.10 0.0.1-SNAPSHOT --transitive --repository file:/Users/axel/.m2/repository

Marking sonymobile:prometheus-relation-model_2.10:0.0.1-SNAPSHOT for download
Preparing to fetch from:
-> file:/var/folders/28/5mj8jbrd13z_nssxk35nd25c0000gn/T/toree_add_deps967503215947550499/
-> file:/Users/axel/.m2/repository
-> https://repo1.maven.org/maven2
-> New file at /Users/axel/.m2/repository/se/lth/cs/nlp/docforia/1.0-SNAPSHOT/docforia-1.0-SNAPSHOT.jar
-> New file at /Users/axel/.m2/repository/junit/junit/4.11/junit-4.11.jar
-> New file at /Users/axel/.m2/repository/org/hamcrest/hamcrest-core/1.3/hamcrest-core-1.3.jar
-> New file at /Users/axel/.m2/repository/com/google/protobuf/protobuf-java/2.6.1/protobuf-java-2.6.1.jar
-> New file at /var/folders/28/5mj8jbrd13z_nssxk35nd25c0000gn/T/toree_add_deps967503215947550499/https/repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-annotations/2.8.6/jackson-annotations-2.8.6.jar
-> New file at /var/folders/28/5mj8jbrd13z_nssxk35nd25c0000gn/T/toree_add_deps967503215947550499/https/repo1.maven.org/maven2/com/fasterxml/jackson/c

In [4]:
import com.sony.relationmodel.CorpusReader

val docs = CorpusReader.readCorpus("/Users/axel/utveckling/prometheus/wikipedia-corpus-herd", 0.001)(sqlContext, sc)

In [28]:
val someDocs = docs.filter(d => d.text().contains("Han hade redan på 1970-talet varit kommunalråd för skolfrågor i samma kommun och ordförande i Katrineholms Arbetarkommun."))

In [29]:
println(someDocs.first)

Name: java.lang.UnsupportedOperationException
Message: empty collection
StackTrace: org.apache.spark.rdd.RDD$$anonfun$first$1.apply(RDD.scala:1344)
org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:150)
org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:111)
org.apache.spark.rdd.RDD.withScope(RDD.scala:316)
org.apache.spark.rdd.RDD.first(RDD.scala:1341)
$line56.$read$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC.<init>(<console>:36)
$line56.$read$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC.<init>(<console>:41)
$line56.$read$$iwC$$iwC$$iwC$$iwC$$iwC.<init>(<console>:43)
$line56.$read$$iwC$$iwC$$iwC$$iwC.<init>(<console>:45)
$line56.$read$$iwC$$iwC$$iwC.<init>(<console>:47)
$line56.$read$$iwC$$iwC.<init>(<console>:49)
$line56.$read$$iwC.<init>(<console>:51)
$line56.$read.<init>(<console>:53)
$line56.$read$.<init>(<console>:57)
$line56.$read$.<clinit>(<console>)
$line56.$eval$.<init>(<console>:7)
$line56.$eval$.<clinit>(<console>)
$line56.$eval.$print(<console>)
sun.

In [16]:
import org.apache.spark.mllib.linalg.{SparseVector, Vector, Vectors}
val sparseV = Vectors.sparse(50, Array(0, 2), Array(1.0, 3.0))
println(sparseV.getClass)

class org.apache.spark.mllib.linalg.SparseVector


# One-vs-All

In [3]:
import java.util.concurrent.TimeUnit.{NANOSECONDS => NANO}

import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.ml.classification.{OneVsRest, LogisticRegression}
import org.apache.spark.ml.util.MetadataUtils
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.sql.DataFrame

In [4]:
val inputData = sqlContext.read.format("libsvm").load("iris.scale")
val data = inputData.randomSplit(Array(0.8, 0.2))

val Array(train, test) = data.map(_.cache())

In [5]:
val classifier = new LogisticRegression().setMaxIter(10).setTol(1E-6).setFitIntercept(true)
inputData.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = false)



In [6]:
val ovr = new OneVsRest()
ovr.setClassifier(classifier)

oneVsRest_6c70a1c26a7e

In [7]:
val ovrModel = ovr.fit(train)

In [11]:
val predictions = ovrModel.transform(test)
val predictionsAndLabels = predictions.select("prediction", "label").map(row => (row.getDouble(0), row.getDouble(1)))
predictions.show(100)
predictions.take(1)(0).get(1)

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.0|(4,[0,1,2,3],[0.0...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|       0.0|
|  0.0|(4,[0,1,2,3],[0.8...|       0.0|
|  0.0|(4,[0,2,3],[0.611...|       0.0|
|  1.0|(4,[0,1,2,3],[-0....|       1.0|
|  1.0|(4,[0,1,2,3],[-0....|       1.0|
|  1.0|(4,[0,1,2,3],[-0....|       1.0|
|  1.0|(4,[0,1,2,3],[-0....|       1.0|
|  2.0|(4,[0,1,2,3],[-0....|       2.0|
|  2.0|(4,[0,1,2,3],[-0....|       2.0|
|  2.0|(4,[0,1,2,3],[-0....|       2.0|
|  0.0|(4,[0,1,2,3],[-0....|       0.0|
|  0.0|(4,[0,1,2,3],[-1....|       0.0|
|  0.0|(4,[0,1,2,3],[0.3...|       0.0|
|  0.0|(4,[0,1,2,3],[0.8...|       0.0|
|  0.0|(4,[0,2,3],[0.222...|       0.0|
|  1.0|(4,[0,1,2,3],[-0....|       1.0|
|  1.0|(4,[0,1,2,3],[-0....|       1.0|
|  1.0|(4,[0,2,3],[-0.77...|       1.0|
|  2.0|(4,[0,1,2,3],[-0....|       2.0|
|  2.0|(4,[0,1,2,3],[-0....|       2.0|


(4,[0,1,2,3],[0.0555554,0.166667,0.491525,0.833333])

In [83]:
val metrics = new MulticlassMetrics(predictionsAndLabels)
val confusionMatrix = metrics.confusionMatrix

In [58]:
println(s" Confusion Matrix\n ${confusionMatrix.toString}\n")

 Confusion Matrix
 10.0  0.0  1.0   
0.0   3.0  0.0   
0.0   0.0  10.0  

